In [ ]:
import pandas as pd

## 1. Reading and Preprocessing

### 1.1 Reading sales data

In [ ]:
input_dir = "../input/m5-forecasting-accuracy/"
data = pd.read_csv(input_dir + "sales_train_validation.csv")
data

In [ ]:
data.columns

### 1.2 Reading calendar data

In [ ]:
calendar_data = pd.read_csv(input_dir + "calendar.csv")
calendar_data

In [ ]:
sale_dates = calendar_data['date']
sale_dates

### 1.3 Columns renaming

In [ ]:
#from datetime import datetime  
#from datetime import timedelta  

#sale_date = datetime(2011,1,29) #First Date in the Sales Calendar
#print(sale_date.date())

In [ ]:
i = 0
for column in enumerate(data.columns):
    #print(i, column)
    if i > 5:
        sale_date = sale_dates[i-6]
        #print(sale_date)
        data.rename(columns={data.columns[column[0]]: sale_date}, inplace= True) 
    i += 1
print(data.columns)

## 2 Predicting data using Same Day Means (without Training any Statistic Model)

Introduction Notes:

"You are predicting 28 forecast days (F1-F28) of items sold for each row. For the validation rows, this corresponds to d_1914 - d_1941, and for the evaluation rows, this corresponds to d_1942 - d_1969"

Validation: Days from 2016-06-20 to 2016-07-17
Evaluation: Days from 2016-07-18 to 2016-08-14

### 2.1 Defining auxiliar and Same Day Means functions

In [ ]:
def get_units_sold(data, month_day):
    columns = list(data.columns)
    #print(columns)
    col = []
    for column in columns:
        if month_day in column:
            col.append(column)
    columns = col
    #print(columns)
    units_sold = data.loc[:,columns]
    #print(units_sold)
    return units_sold

In [ ]:
def get_mean(units_sold, index):
    #print(units_sold.loc[index,:])
    sum_units = sum(units_sold.loc[index,:])
    units = len(units_sold.loc[index,:])
    result = sum_units//units
    #print(result)
    return result

In [ ]:
def get_list_means(units_sold):
    list_means = []
    for i in range(len(units_sold)):
        list_means.append(get_mean(units_sold, i))
    return list_means

In [ ]:
#units_sold = get_units_sold(data,"07-04") #Example Independence Day
#get_mean(units_sold, 2186) #Example Item with index 2186
#print(get_list_means(units_sold)) #List of all Items Means by Same Day

### 2.2 Predicting data

#### 2.2.1 Predicting Validation data

In [ ]:
validation_dates = calendar_data.loc[1913:1940,'date']
prediction_validation = {}
for validation_date in list(validation_dates):
    #print(validation_date)
    month_day = validation_date[5:]
    #print(month_day)
    units_sold = get_units_sold(data, month_day)
    list_means = get_list_means(units_sold)
    prediction_validation[validation_date] = list_means
print(prediction_validation)

In [ ]:
#print(prediction_validation)

#### 2.2.2 Predicting Evaluation Data

In [ ]:
evaluation_dates = calendar_data.loc[1941:1969,'date']
prediction_evaluation = {}
for evaluation_date in list(evaluation_dates):
    #print(validation_date)
    month_day = evaluation_date[5:]
    #print(month_day)
    units_sold = get_units_sold(data, month_day)
    list_means = get_list_means(units_sold)
    prediction_evaluation[evaluation_date] = list_means
print(prediction_evaluation)

In [ ]:
#print(prediction_evaluation)

### 2.3 Building, Concatenating, and Writing Prediction DataFrames to CSV

#### 2.3.1 Building Validation Prediction Dataframe

In [ ]:
final_columns = ['F1','F2','F3','F4','F5','F6','F7','F8','F9','F10','F11','F12','F13','F14','F15','F16','F17'
           ,'F18','F19','F20','F21','F22','F23','F24','F25','F26','F27','F28']
columns = list(prediction_validation.keys())
ids = data['id']
#print(ids)
df = pd.DataFrame(prediction_validation, columns = columns)
i = 0
for column in enumerate(df.columns):
    df.rename(columns={df.columns[column[0]]: final_columns[i]}, inplace= True)
    i += 1
df.insert(0, 'id', ids)
print(df)

#### 2.3.2 Building Evaluation Prediction Dataframe

In [ ]:
ids2 = list(ids)
for i in range(len(ids2)):
    ids2[i] = ids2[i].replace("validation","evaluation")
#print(ids2)
columns = list(prediction_evaluation.keys())
df2 = pd.DataFrame(prediction_evaluation, columns = columns)
i = 0
for column in enumerate(df2.columns):
    df2.rename(columns={df2.columns[column[0]]: final_columns[i]}, inplace= True)
    i += 1
df2.insert(0, 'id', ids2)
print(df2)

#### 2.3.3 Concatenating DataFrames

In [ ]:
final_df = pd.concat([df, df2], ignore_index=True)
final_df

#### 2.3.4 Writing to CSV

In [ ]:
final_df.to_csv("submission.csv", index = False)